In [6]:
import os
import json
import glob
import pandas as pd
import numpy as np
from pymoo.indicators.igd import IGD

def calculate_igd(front, reference_pareto):
    """
    Calculate IGD between the front and the reference Pareto front.
    """
    igd_indicator = IGD(reference_pareto)
    return igd_indicator(front)

# Settings
base_path = "F:/setup files/Downloads/ResultLastly/"
data_folder = os.path.join(base_path)  # Adjust if your JSON files are in a subfolder
nadir_csv_path = os.path.join(base_path, "nadir_points.csv")
pareto_fronts_path = os.path.join(base_path, "global_pareto_fronts_last_generation.json")

# Load the nadir points CSV and precomputed Pareto fronts JSON
nadir_df = pd.read_csv(nadir_csv_path)
with open(pareto_fronts_path, 'r') as pf_file:
    pareto_fronts_data = json.load(pf_file)

# Define the target generation
target_generation = "100"

# Initialize a list to store results
results = []

# Get all JSON files in the data folder
json_files = glob.glob(os.path.join(data_folder, "*.json"))

if not json_files:
    print("No JSON files found in the specified data folder.")
else:
    for file_path in json_files:
        file_name = os.path.basename(file_path)

        # Retrieve nadir information corresponding to the file
        nadir_row = nadir_df[nadir_df['file'] == file_name]
        if nadir_row.empty:
            print(f"Nadir info not found for {file_name}. Skipping.")
            continue
        nadir_row = nadir_row.iloc[0]

        # Identify objective columns for nadir points
        obj_columns = sorted(
            [col for col in nadir_df.columns if col.startswith("obj_") and col.endswith("_nadir")],
            key=lambda x: int(x.split('_')[1])
        )

        # Extract the nadir vector
        nadir_vector = np.array([nadir_row[col] for col in obj_columns])

        # Retrieve the reference Pareto front from the precomputed data for this file
        reference_points = pareto_fronts_data.get(file_name, [])
        reference_points = np.array(reference_points)

        if reference_points.size == 0:
            print(f"No reference Pareto points found for {file_name}. Skipping IGD computation.")
            continue

        # Scale the reference Pareto front by the nadir vector
        scaled_ref = reference_points / nadir_vector

        # Load JSON data from file
        with open(file_path, 'r') as f:
            data = json.load(f)

        # Iterate over each algorithm in the JSON data
        for algorithm in data:
            if target_generation not in data[algorithm]:
                print(f"Generation {target_generation} not found for {algorithm} in {file_name}. Skipping.")
                continue

            solutions = data[algorithm][target_generation]
            solutions_np = np.array(solutions)

            if solutions_np.size == 0:
                print(f"No solutions found for {algorithm} at generation {target_generation} in {file_name}. Skipping.")
                continue

            # Scale the solutions using the nadir vector
            scaled_solutions = solutions_np / nadir_vector

            try:
                # Compute IGD on the scaled solutions using the scaled reference Pareto front
                igd_value = calculate_igd(scaled_solutions, scaled_ref)
            except Exception as e:
                print(f"Error computing IGD for {algorithm} in {file_name}: {e}")
                igd_value = np.nan

            # Append the result
            results.append({
                "Instance": file_name,
                "Algorithm": algorithm,
                "IGD": igd_value
            })

    # Convert results to a DataFrame
    df_results = pd.DataFrame(results)

    if df_results.empty:
        print("No IGD results were computed.")
    else:
        # Pivot the DataFrame to have algorithms as columns
        pivot_df = df_results.pivot(index="Instance", columns="Algorithm", values="IGD")

        # Calculate mean and std for each algorithm
        mean_series = pivot_df.mean(axis=0)
        std_series = pivot_df.std(axis=0)

        # Combine instance-wise IGD and summary
        combined_df = pivot_df.copy()
        combined_df.loc["Mean"] = mean_series
        combined_df.loc["Std"] = std_series

        # Optionally, save detailed and summary results to CSV
        detailed_csv = os.path.join(base_path, "igd_pivot.csv")
        combined_df.to_csv(detailed_csv, index_label='Instance')

        print(f"Detailed IGD results saved to {detailed_csv}")

Nadir info not found for global_pareto_fronts_last_generation.json. Skipping.
Detailed IGD results saved to F:/setup files/Downloads/ResultLastly/igd_pivot.csv
